In [1]:
import numpy as np
#import jax.numpy as jnp
data = np.loadtxt("/var/home/luka/proj/Papilonidae_dataset_v2/Papilionidae_aligned_new.txt", delimiter="\t").reshape((2240, 200))


In [29]:
from ete3 import Tree, TreeStyle, Tree, TextFace, add_face_to_node
# Load the tree and get the leaf order in inorder traversal
ptree = Tree("/var/home/luka/proj/Papilonidae_dataset_v2/papilionidae_tree.txt", format=1)
order = ptree.get_leaf_names()  # returns in-order leaves (same as pre-order when just looking at leaves)

ind_dict = {}

i = 0
for n in ptree.traverse("preorder"):
    ind_dict[n.name] = i
    i += 1

ts = TreeStyle()
ts.show_leaf_name = False
ts.show_scale = False
def my_layout(node):
        F = TextFace(str(ind_dict[node.name]),ftype="Courier",fsize=8)
        add_face_to_node(F, node, column=0, position="branch-right")
ts.layout_fn = my_layout
ptree.render(file_name="tree-short.pdf",tree_style=ts,w=1040,h=2160)
#ptree.show(tree_style=ts)


{'nodes': [[0.9252669039145908,
   220.70728125,
   8.327402135231317,
   227.61928125,
   0,
   None],
  [216.54144560177662, 20.736, 223.94358083309334, 27.648, 1, None],
  [79.74810217146893, 420.6785625, 87.15023740278565, 427.5905625, 2, None],
  [180.3461156072587, 130.248, 187.74825083857542, 137.16, 3, None],
  [302.0561589994704, 65.664, 309.45829423078715, 72.576, 4, None],
  [224.25039257407806, 194.832, 231.65252780539478, 201.744, 5, None],
  [320.1302695520475, 144.288, 327.5324047833642, 151.2, 6, None],
  [424.56584559062657, 110.592, 431.96798082194334, 117.504, 7, None],
  [364.62159575555415, 177.984, 372.0237309868709, 184.896, 8, None],
  [453.65256497217143, 155.52, 461.05470020348815, 162.432, 9, None],
  [450.6303485314478,
   200.448,
   458.03248376276457,
   207.35999999999999,
   10,
   None],
  [354.1682539977515, 245.376, 361.5703892290682, 252.288, 11, None],
  [131.5123161406019, 711.109125, 138.91445137191863, 718.021125, 12, None],
  [213.1387342854222

In [30]:
import pandas as pd

# Load categories
categories = pd.read_csv("/var/home/luka/proj/Papilonidae_dataset_v2/Papilonidae_metadata_new.txt", header=None)[0]

# Create DataFrame from data
df = pd.DataFrame(data.reshape(2240, -1))
df['category'] = pd.Categorical(categories, categories=categories.unique())

# Group by category and calculate means
means = df.groupby('category', observed=True).mean()

# Add order column based on the tree order
df['order'] = pd.Categorical(df['category'], categories=order, ordered=True)


# Reorder means DataFrame based on the order column
means = means.reset_index()
means['order'] = pd.Categorical(means['category'], categories=order, ordered=True)
means = means.sort_values('order')

# Drop unnecessary columns and convert to numpy array
X = means.drop(columns=['category', 'order']).values

N, d = X.shape

In [31]:
D = np.zeros((N*d, d))

# Create an array of indices
i_indices = np.arange(N*d)
j_indices = np.arange(d)

# Use broadcasting to create a mask
mask = (j_indices[:, None] * N <= i_indices) & (i_indices < (j_indices[:, None] + 1) * N)
D[mask.T] = 1.0


In [22]:
preorder = [n for n in ptree.traverse("preorder")]

for n in preorder:
    print(f'{n.name:<35}{n.dist}')

QS1803                             0.0
Baronia_brevicornis                0.1711
QS1804                             0.04527
QS1807                             0.0653
Iphiclides_podalirius              0.08472
QS1808                             0.01315
QS1810                             0.06096
Graphium_evemon                    0.06883
QS1811                             0.01369
Graphium_sarpedon                  0.05466
Graphium_agamemnon                 0.05188
Protographium_marcellus            0.09227
QS1818                             0.02038
QS1819                             0.02572
QS1820                             0.06041
Hypermnestra_helios                0.04444
QS1821                             0.0276
Parnassius_orleans                 0.01792
Parnassius_honrathi                0.03123
QS1825                             0.01972
QS1826                             0.03388
Archon_apollinus                   0.03581
Luehdorfia_puziloi                 0.03646
QS1828            

In [33]:
preorder = [n for n in ptree.traverse("preorder")]
M = len(preorder)   # number of nodes in tree (including internal)
dists = np.zeros((M))
inds = np.zeros(N, dtype=int)
inds_r = np.zeros(M, dtype=int)



j = 0
for i in range(M):
    n = preorder[i]
    branch_size = len(n.get_descendants())+1
    dists[i:i+branch_size] += n.dist
    if n.name[0] != 'Q':
        inds[j] = i
        inds_r[i] = j
        j += 1

leaves = ptree.get_leaves()
Cov = np.zeros((N, N))

for i in range(N):
    for j in range(i, N):
        ancestor = leaves[i].get_common_ancestor(leaves[j])
        lca_ij = preorder.index(ancestor)
        Cov[i, j] = Cov[j, i] = dists[lca_ij]


In [34]:
lca_A = np.zeros((N, N), dtype=int)

for i in range(N):
    for j in range(i, N):
        ancestor = leaves[i].get_common_ancestor(leaves[j])
        lca_ij = preorder.index(ancestor)
        lca_A[i, j] = lca_A[j, i] = lca_ij


In [37]:
lca_a = np.zeros((M, M), dtype=int)

for i in range(M):
    for j in range(i, M):
        ancestor = preorder[i].get_common_ancestor(preorder[j])
        lca_ij = preorder.index(ancestor)
        lca_a[i, j] = lca_a[j, i] = lca_ij


In [60]:
lca_b = np.zeros((M, M), dtype=int)
sizes = np.zeros(M, dtype=int)

def get_size(i):
    s = sizes[i]
    if s == 0:
        s = len(preorder[i].get_descendants())+1
        sizes[i] = s
    return s

for i in range(M):
    node = preorder[i]
    branch_size = get_size(i)

    if branch_size >= 2:
        l = i+1
        r = l+get_size(l)
        end = r+get_size(r)
        lca_b[i:r,r:end] = i
        lca_b[r:end,i:r] = i

        lca_b[i,i:r] = lca_b[i:r,i] = i

        #print(f'{i:<10}l:{l:<10}r:{r:<10}end:{end}')
    else: print(f'{i:<10}{branch_size}')
    lca_b[i,i] = i

1         1
4         1
7         1
9         1
10        1
11        1
15        1
17        1
18        1
21        1
22        1
24        1
26        1
27        1
29        1
33        1
34        1
36        1
40        1
42        1
43        1
46        1
48        1
50        1
51        1
52        1
55        1
57        1
58        1
61        1
62        1
63        1
68        1
70        1
71        1
73        1
75        1
76        1
78        1
81        1
83        1
84        1
86        1
88        1
90        1
92        1
93        1
94        1


In [59]:
print(f'{"i":<5}{"j":<5}{"a":<5}{"b":<5}')
for i in range(M):
    for j in range(i, M):
        a = lca_a[i, j]
        b = lca_b[i, j]

        if a != b:
            print(f'{i:<5}{j:<5}{a:<5}{b:<5}')

i    j    a    b    


In [25]:
v1 = np.ones(N)
Cov_inv = np.linalg.inv(Cov)
tmp = v1.T @ Cov_inv
mle_r = ((tmp @ v1) **-1) * (tmp @ X)
assert mle_r.shape==(d,)

tmp = X - mle_r.T
mle_R = (((N - 1) ** -1) * tmp.T) @ Cov_inv @ tmp
assert mle_R.shape==(d,d)



In [8]:
not []

True

In [13]:
#X_mean = np.mean(X,axis=0)
evals, evecs = np.linalg.eigh(mle_R)
X_cent = X - mle_r[None,:]

In [9]:


def ppca_recon(k=2):
    V_k = evecs[:, -k:]
    #print(X_cent.shape,V_k.shape)
    X_reduced = X_cent @ V_k
    #print(X_reduced.shape)
    X_reconstructed = X_reduced @ V_k.T + mle_r[None,:]
    #print(X_reconstructed.shape)
    return X_reconstructed

runtime modul

In [10]:
for k in range(1,N+1):
    ppca_recon(k)